In [5]:
import warnings
warnings.filterwarnings('ignore')
import glob
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
import random

In [6]:
!pip install category_encoders
import category_encoders as ce

     |████████████████████████████████| 81kB 2.4MB/s 


In [7]:
import copy

In [26]:
train = pd.read_csv('train.csv')
train = train.drop(['index'], axis=1)
train.fillna('NAN', inplace=True) 


test = pd.read_csv('test.csv')
test = test.drop(['index'], axis=1)
test.fillna('NAN', inplace=True)

submit = pd.read_csv('sample_submission.csv')

In [9]:
object_col = []
for col in train.columns:
    if train[col].dtype == 'object':
        object_col.append(col)

In [12]:
for i in object_col:
  print("{} : {}".format(i,len(train[i].unique())))

gender : 2
car : 2
reality : 2
income_type : 5
edu_type : 5
family_type : 5
house_type : 6
occyp_type : 19


In [4]:
object_col

['gender',
 'car',
 'reality',
 'income_type',
 'edu_type',
 'family_type',
 'house_type',
 'occyp_type']

In [28]:
y

,credit
0,1.0
1,1.0
2,2.0
3,0.0
4,2.0
...,...
26452,1.0
26453,2.0
26454,2.0
26455,2.0


In [29]:
X=train.drop("credit",axis=1).reset_index(drop=True)
y=pd.DataFrame(train["credit"]).reset_index(drop=True)

onehot_cols=['gender','car','reality','income_type','edu_type','family_type']
enc = ce.OneHotEncoder(cols=onehot_cols)
train_onehot=enc.fit_transform(X)
test_onehot =enc.transform(test)



# train_onehot_df = pd.DataFrame(enc.transform(train.loc[:,object_col]).toarray(), 
#              columns=enc.get_feature_names(object_col))
# train.drop(object_col, axis=1, inplace=True)
# train = pd.concat([train, train_onehot_df], axis=1)



In [30]:
print(train_onehot)

       gender_1  gender_2  car_1  ...      occyp_type  family_size  begin_month
0             1         0      1  ...             NAN          2.0         -6.0
1             1         0      1  ...        Laborers          3.0         -5.0
2             0         1      0  ...        Managers          2.0        -22.0
3             1         0      1  ...     Sales staff          2.0        -37.0
4             1         0      0  ...        Managers          2.0        -26.0
...         ...       ...    ...  ...             ...          ...          ...
26452         1         0      1  ...      Core staff          4.0         -2.0
26453         1         0      1  ...             NAN          2.0        -47.0
26454         1         0      0  ...      Core staff          2.0        -25.0
26455         0         1      1  ...        Laborers          1.0        -59.0
26456         1         0      1  ...  Security staff          2.0         -9.0

[26457 rows x 33 columns]


In [31]:

hash_cols=['house_type','occyp_type']
ce_hash=ce.HashingEncoder(cols=hash_cols)
train_hash=ce_hash.fit_transform(train_onehot)
test_hash =ce_hash.transform(test_onehot)



In [46]:
train_hash

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,gender_1,gender_2,car_1,car_2,reality_1,reality_2,child_num,income_total,income_type_1,income_type_2,income_type_3,income_type_4,income_type_5,edu_type_1,edu_type_2,edu_type_3,edu_type_4,edu_type_5,family_type_1,family_type_2,family_type_3,family_type_4,family_type_5,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month
0,0,1,0,0,0,1,0,0,1,0,1,0,1,0,0,202500.0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,-13899,-4709,1,0,0,0,2.0,-6.0
1,0,0,0,0,1,1,0,0,1,0,1,0,0,1,1,247500.0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,-11380,-1540,1,0,0,1,3.0,-5.0
2,0,1,0,0,1,0,0,0,0,1,0,1,0,1,0,450000.0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,-19087,-4434,1,0,1,0,2.0,-22.0
3,0,0,0,1,1,0,0,0,1,0,1,0,0,1,0,202500.0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,-15088,-2092,1,0,1,0,2.0,-37.0
4,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,157500.0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,-15037,-2105,1,0,0,0,2.0,-26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,0,0,0,0,1,0,0,1,1,0,1,0,1,0,2,225000.0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,-12079,-1984,1,0,0,0,4.0,-2.0
26453,0,0,0,0,1,1,0,0,1,0,1,0,0,1,1,180000.0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,-15291,-2475,1,0,0,0,2.0,-47.0
26454,0,0,0,0,0,1,0,1,1,0,0,1,1,0,0,292500.0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,-10082,-2015,1,0,0,0,2.0,-25.0
26455,0,0,0,0,1,1,0,0,0,1,1,0,0,1,0,171000.0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,-10145,-107,1,0,0,0,1.0,-59.0


In [38]:


train=pd.concat([train_hash,y],axis=1)

In [37]:
print(train_df)

       col_0  col_1  col_2  col_3  ...  email  family_size  begin_month  credit
0          0      1      0      0  ...      0          2.0         -6.0     1.0
1          0      0      0      0  ...      1          3.0         -5.0     1.0
2          0      1      0      0  ...      0          2.0        -22.0     2.0
3          0      0      0      1  ...      0          2.0        -37.0     0.0
4          0      1      0      0  ...      0          2.0        -26.0     2.0
...      ...    ...    ...    ...  ...    ...          ...          ...     ...
26452      0      0      0      0  ...      0          4.0         -2.0     1.0
26453      0      0      0      0  ...      0          2.0        -47.0     2.0
26454      0      0      0      0  ...      0          2.0        -25.0     2.0
26455      0      0      0      0  ...      0          1.0        -59.0     2.0
26456      0      0      0      0  ...      0          2.0         -9.0     2.0

[26457 rows x 40 columns]


In [22]:
print(train.shape)
print(test_hash.shape)

(52914, 40)
(10000, 39)


In [ ]:
# test_onehot_df = pd.DataFrame(enc.transform(test.loc[:,object_col]).toarray(), 
#              columns=enc.get_feature_names(object_col))
# test.drop(object_col, axis=1, inplace=True)
# test = pd.concat([test, test_onehot_df], axis=1)

In [40]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in skf.split(train, train['credit']):
    folds.append((train_idx, valid_idx))

In [41]:
random.seed(42)
lgb_models={}
for fold in range(5):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                         train['credit'][train_idx].values, train['credit'][valid_idx].values 
    lgb = LGBMClassifier(n_estimators=1000)
    lgb.fit(X_train, y_train, 
            eval_set=[(X_train, y_train), (X_valid, y_valid)], 
            early_stopping_rounds=30,
           verbose=100)
    lgb_models[fold]=lgb
    print(f'================================================================================\n\n')

====================================1============================================
Training until validation scores don't improve for 30 rounds.
[100]	training's multi_logloss: 0.679828	valid_1's multi_logloss: 0.756898
[200]	training's multi_logloss: 0.606024	valid_1's multi_logloss: 0.737039
[300]	training's multi_logloss: 0.548345	valid_1's multi_logloss: 0.726016
[400]	training's multi_logloss: 0.500459	valid_1's multi_logloss: 0.72218
Early stopping, best iteration is:
[434]	training's multi_logloss: 0.486981	valid_1's multi_logloss: 0.721703


====================================2============================================
Training until validation scores don't improve for 30 rounds.
[100]	training's multi_logloss: 0.675004	valid_1's multi_logloss: 0.77255
[200]	training's multi_logloss: 0.600541	valid_1's multi_logloss: 0.759005
[300]	training's multi_logloss: 0.543063	valid_1's multi_logloss: 0.753268
[400]	training's multi_logloss: 0.496813	valid_1's multi_logloss: 0.752826
Ea

In [43]:
submit.iloc[:,1:]=0
for fold in range(5):
    submit.iloc[:,1:] += lgb_models[fold].predict_proba(test_hash)/5

In [45]:
submit.to_csv('20210513_test_submit_lightgbm.csv', index=False) # 0.7272812144